In [9]:
%matplotlib inline
import os
import pickle
import json
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy.stats as stats
import dotenv
dotenv.load_dotenv()

True

In [10]:
plt.style.use('../style/style-formal.mplstyle')

In [143]:
RESEARCH_DATA = os.environ.get("RESEARCH_DATA")

EL = "attempt_1"
PIDS = [
    "NSFW_c_frw",
    "NSFW_c_timer",
    "NSFW_c_msg",
    "NSFW_cpp_cfg",
    "NSFW_cpp_file",
    "NSFW_cpp_thread"
]

STD_DIR = os.path.join(RESEARCH_DATA, f"{EL}/dlfl_out/experiment_raw_results_All")
CMP_DIR = os.path.join(RESEARCH_DATA, f"{EL}/dlfl_out/experiment_raw_results_All")
assert os.path.exists(STD_DIR), f"Standard result directory does not exist: {STD_DIR}"
assert os.path.exists(CMP_DIR), f"Comparison result directory does not exist: {CMP_DIR}"

COMPARISON = [
    (STD_DIR, "lineCnt50_mutCnt10_tcsReduced"),
    (CMP_DIR, "lineCnt50_mutCnt1_tcsReduced"),
]

REPEAT_CNT = 10
K_FOLD = 10


In [144]:

def get_ranks(CMP_INFO):
    rid2bid2ranks = {}
    for rid in range(1, REPEAT_CNT+1):
        method_dir = os.path.join(CMP_INFO[0], f"repeat_{rid}/methods/{CMP_INFO[1]}")
        assert os.path.exists(method_dir), f"Method directory does not exist: {method_dir}"
        rid2bid2ranks[rid] = {}
        for kid in range(1, K_FOLD+1):
            result_file = os.path.join(method_dir, f"results/result_{kid}.txt")
            assert os.path.exists(result_file), f"Result file does not exist: {result_file}"

            with open(result_file, 'r') as f:
                lines = f.readlines()
                line_id = 0
                while line_id < len(lines):
                    line = lines[line_id].strip()
                    if line.startswith("==="):
                        bid_name = line.strip().split("===")[-1].strip()
                        if bid_name not in rid2bid2ranks[rid]:
                            rid2bid2ranks[rid][bid_name] = []
                        line_id += 1
                        while line_id < len(lines):
                            new_line = lines[line_id].strip()
                            if new_line.startswith("==="):
                                line_id -= 1
                                break
                            rank = int(new_line.strip())
                            rid2bid2ranks[rid][bid_name].append(rank)
                            line_id += 1
                    line_id += 1
    return rid2bid2ranks


In [145]:
STD_BID2RANKS = get_ranks(COMPARISON[0])
CMP_BID2RANKS = get_ranks(COMPARISON[1])

In [146]:
BID_LIST = list(STD_BID2RANKS[1].keys())
std_rank_list = []
cmp_rank_list = []
for bid in BID_LIST:
    for rid in range(1, REPEAT_CNT+1):
        std_rank_list.append(np.mean(STD_BID2RANKS[rid][bid]))
        cmp_rank_list.append(np.mean(CMP_BID2RANKS[rid][bid]))
print(f"STD LEN: {len(std_rank_list)}")
print(f"CMP LEN: {len(cmp_rank_list)}")

STD LEN: 3000
CMP LEN: 3000


In [147]:
wilcoxon_stat, p_value = stats.wilcoxon(
    x=std_rank_list,
    y=cmp_rank_list,
    zero_method='zsplit',
    alternative='two-sided'
)

In [148]:
print(f"wilcoxon_stat: {wilcoxon_stat}")
print(f"p_value: {p_value}")

wilcoxon_stat: 1843499.5
p_value: 1.6763268651989609e-19
